In [6]:
pip install opencv-python


  Using cached opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl (38.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install dlib


  Using cached dlib-19.24.2.tar.gz (11.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.2-cp310-cp310-win_amd64.whl size=2846266 sha256=a2d8375aa0b3a1207363a643928aac20d0c1f80fa663c8bd3c82ba8c8dafb5bb
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\bc\19\6e\7461e0584c10c486b31b7d204a591d5e35812c13901ae93c24
Successfully built dlib


In [10]:
pip install imutils

  Using cached imutils-0.5.4.tar.gz (17 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25854 sha256=5d87ddc44d89a19fb56b46d774f402d4aae67aef368fbfed8ac34410b18ca548
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\c2\02\32\f3617a9f68bcc67eda3ebeb4514eba18f62e81ff439428109d
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from os.path import dirname, realpath, sep, pardir
import sys
sys.path.append(dirname(realpath(sys.executable)) + sep + pardir + sep + "lib")
import cv2
import numpy as np
import dlib
from imutils import face_utils

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

sleep = 0
drowsy = 0
active = 0
status = ""
color = (0,0,0)

def compute(ptA, ptB):
    return np.linalg.norm(ptA-ptB)

def blinked(a,b,c,d,e,f):
    up = compute(b,d) + compute(c,e)
    down = compute(a,f)
    ratio = up/(2.0*down)
    
    if(ratio > 0.25):
        return 2
    elif(ratio > 0.21 and ratio<=0.25):
        return 1
    else:
        return 0


while True:
    rt, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        
        face_frame = frame.copy()
        cv2.rectangle(face_frame, (x1,y1), (x2,y2), (0,255,0), 2)
        
        landmarks = predictor(gray,face)
        landmarks = face_utils.shape_to_np(landmarks)
        
        left_blink = blinked(landmarks[36],landmarks[37],landmarks[38],landmarks[41],landmarks[40],landmarks[39])
        right_blink = blinked(landmarks[42],landmarks[43],landmarks[44],landmarks[47],landmarks[46],landmarks[45])
        
        if(left_blink == 0 and right_blink == 0):
            sleep+=1
            drowsy=0
            active=0
            if(sleep>2):
                status = "SLEEPING !!"
                color = (255,0,0)
        
        elif(left_blink == 1 and right_blink == 1):
            sleep=0
            drowsy+=1
            active=0
            if(drowsy>2):
                status = "DROWSY !!"
                color = (0,0,255)
                
        else: 
            drowsy=0
            sleep=0
            active+=1
            if(active>2):
                status = "ACTIVE !!"
                color = (0,255,0)
            
        
        cv2.putText(frame,status,(100,100),cv2.FONT_HERSHEY_SIMPLEX,1.2,color,3)
        
        for n in range(0,68):
            (x,y) = landmarks[n]
            cv2.circle(face_frame,(x,y),1,(255,255,255),-1)
    
        cv2.imshow("Frame",frame)
        cv2.imshow("Result of detector", face_frame)

        key = cv2.waitKey(1)
        if key == 27:
            break

